**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [224]:
!pip install keras-flops

In [225]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [226]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [227]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [228]:
# Преобразуйте метки классов в one_hot формат
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, CLASSES)
y_val = np_utils.to_categorical(y_val, CLASSES)

In [229]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [242]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(16, kernel_size=3,strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Conv2D(16, kernel_size=3,strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
    
    
])

In [243]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6820e6


In [244]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_74"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_134 (Conv2D)         (None, 16, 16, 16)        448       
                                                                 
 batch_normalization_207 (Ba  (None, 16, 16, 16)       64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_69 (MaxPoolin  (None, 8, 8, 16)         0         
 g2D)                                                            
                                                                 
 activation_281 (Activation)  (None, 8, 8, 16)         0         
                                                                 
 conv2d_135 (Conv2D)         (None, 4, 4, 16)          2320      
                                                                 
 batch_normalization_208 (Ba  (None, 4, 4, 16)       

In [245]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [246]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=50
)

Epoch 1/50
391/391 [==============================] - 9s 19ms/step - loss: 3.6714 - accuracy: 0.1576 - val_loss: 3.6865 - val_accuracy: 0.1618
Epoch 2/50
391/391 [==============================] - 7s 18ms/step - loss: 3.0779 - accuracy: 0.2449 - val_loss: 3.0235 - val_accuracy: 0.2591
Epoch 3/50
391/391 [==============================] - 7s 18ms/step - loss: 2.8401 - accuracy: 0.2881 - val_loss: 2.8459 - val_accuracy: 0.2960
Epoch 4/50
391/391 [==============================] - 7s 18ms/step - loss: 2.6710 - accuracy: 0.3248 - val_loss: 2.6961 - val_accuracy: 0.3250
Epoch 5/50
391/391 [==============================] - 7s 18ms/step - loss: 2.5357 - accuracy: 0.3522 - val_loss: 2.5862 - val_accuracy: 0.3487
Epoch 6/50
391/391 [==============================] - 7s 18ms/step - loss: 2.4399 - accuracy: 0.3717 - val_loss: 2.5381 - val_accuracy: 0.3619
Epoch 7/50
391/391 [==============================] - 7s 18ms/step - loss: 2.3534 - accuracy: 0.3897 - val_loss: 2.5264 - val_accuracy: 0.3622